## IMPORTS

In [1]:
import requests
from bs4 import BeautifulSoup
import requests
import json
import re
import zipfile
import pyodbc


## GET FIRST SET OF COOKIES

In [2]:
urlIndex = "https://ar.kairosweb.com/"
sesion = requests.Session()
response = sesion.get(urlIndex)

In [3]:
responseHeaders = dict(response.headers)
responseHeadersFormated = json.dumps(responseHeaders, indent=4)

headerCFRAY = responseHeaders["CF-RAY"]
headerCFRAY = headerCFRAY.split("-")[0]

responseCookies = response.cookies
print("responseCookies", responseCookies)
print("sessionCookies", sesion.cookies)

responseBody = response.text
body = BeautifulSoup(responseBody, "html.parser")
securityLogin = body.find("input", {"id": "security-login", "name":"security-login"})
securityLoginValue=securityLogin.attrs["value"]

print(type(securityLoginValue))




responseCookies <RequestsCookieJar[<Cookie __wpdm_client=7afd32acfc33b9c462944c4c86f08906 for ar.kairosweb.com/>, <Cookie _learn_press_session_32928db1270e4fa92127688908034133=c40a6c469559206049c83f6d668e6796%7C%7C1702485781%7C%7Ca6e72517025b73d6ce8896ca4923e4e9 for ar.kairosweb.com/>, <Cookie _wordpress_lp_guest=13739313eb378a273e26121b564ef47d for ar.kairosweb.com/>]>
sessionCookies <RequestsCookieJar[<Cookie __wpdm_client=7afd32acfc33b9c462944c4c86f08906 for ar.kairosweb.com/>, <Cookie _learn_press_session_32928db1270e4fa92127688908034133=c40a6c469559206049c83f6d668e6796%7C%7C1702485781%7C%7Ca6e72517025b73d6ce8896ca4923e4e9 for ar.kairosweb.com/>, <Cookie _wordpress_lp_guest=13739313eb378a273e26121b564ef47d for ar.kairosweb.com/>]>
<class 'str'>


## GET LINK TO SECOND SET OF COOKIES

In [4]:
soup = BeautifulSoup(responseBody, "html.parser")
scripts = soup.find_all("script")

parametro_r=""

for script in scripts:
    if script.string:
        match = re.search(
            r"window\['__CF\$cv\$params'\]=\{r:'(.*?)'", script.string)
        if match:
            parametro_r = match.group(1)
            

In [ ]:
print("parametro_r:", type(parametro_r))
print("parametro_r:", parametro_r)
print("headerCFRAY: ", type(headerCFRAY))
print("headerCFRAY: ", headerCFRAY)


## GET SECOND SET OF COOKIES

In [5]:
if parametro_r == headerCFRAY:
    secondSetCookies =  "https://ar.kairosweb.com/cdn-cgi/challenge-platform/h/b/jsd/r/"+parametro_r
    print("secondSetCookies: ", secondSetCookies)
    response2 = sesion.post(secondSetCookies)
    print("response2: ", response2)
    print("sesion: ", sesion.cookies)

secondSetCookies:  https://ar.kairosweb.com/cdn-cgi/challenge-platform/h/b/jsd/r/833f2f228d77c823
response2:  <Response [200]>
sesion:  <RequestsCookieJar[<Cookie __wpdm_client=7afd32acfc33b9c462944c4c86f08906 for ar.kairosweb.com/>, <Cookie _learn_press_session_32928db1270e4fa92127688908034133=c40a6c469559206049c83f6d668e6796%7C%7C1702485781%7C%7Ca6e72517025b73d6ce8896ca4923e4e9 for ar.kairosweb.com/>, <Cookie _wordpress_lp_guest=13739313eb378a273e26121b564ef47d for ar.kairosweb.com/>, <Cookie cf_clearance=TvlDQZpe.4lZQBrSe8d1eplFv.nssLWGa1YI5ib6eI8-1702312998-0-1-cb20db7e.70214d3b.9c21849e-0.1.1702312998 for .kairosweb.com/>]>


## LOGIN

In [6]:
params = {
    "username": "compras@drogueriacomarsa.com.ar",
    "password": "comarsa+2944",
    "remember-me": "on",
    "redirect_to":"", 
    "lrm_action": "login",
    "wp-submit": "1",
    "lp-ajax": "login",
    "security-login": securityLoginValue,
    "_wp_http_referer": "/"
}

response = sesion.post("https://ar.kairosweb.com/?lrm=1", data=params)
print("response", response.cookies)
print("sesion: ", sesion.cookies)

response <RequestsCookieJar[<Cookie _learn_press_session_32928db1270e4fa92127688908034133=c40a6c469559206049c83f6d668e6796%7C%7C1702485781%7C%7Ca6e72517025b73d6ce8896ca4923e4e9 for ar.kairosweb.com/>, <Cookie wordpress_logged_in_32928db1270e4fa92127688908034133=compras%40drogueriacomarsa.com.ar%7C1703522602%7C2MAV9UgQ94Lt2293GWSEkIY6ppBZkMFHklX3ZTGRbUF%7Cbe44c3caade7f629e0df0a35279c106e0650bd65d0614153724b1171d3b8463a for ar.kairosweb.com/>, <Cookie wordpress_sec_32928db1270e4fa92127688908034133=compras%40drogueriacomarsa.com.ar%7C1703522602%7C2MAV9UgQ94Lt2293GWSEkIY6ppBZkMFHklX3ZTGRbUF%7C59ca0463d3ba31152f674446813bd2da4d9adbeb4aa4abe5f777f4fd56d87d0d for ar.kairosweb.com/wp-content/plugins>, <Cookie wordpress_sec_32928db1270e4fa92127688908034133=compras%40drogueriacomarsa.com.ar%7C1703522602%7C2MAV9UgQ94Lt2293GWSEkIY6ppBZkMFHklX3ZTGRbUF%7C59ca0463d3ba31152f674446813bd2da4d9adbeb4aa4abe5f777f4fd56d87d0d for ar.kairosweb.com/wp-admin>]>
sesion:  <RequestsCookieJar[<Cookie __wpdm_client

## GET TO kairos-base-de-datos

In [7]:
response = sesion.get("https://ar.kairosweb.com/kairos-base-de-datos/")

soup = BeautifulSoup(response.text, "html.parser")
enlaces = soup.find_all("a", class_="wpdm-download-link download-on-click btn btn-primary btn-xs")

print("enlaces:", enlaces)

for enlace in enlaces:
    dateSubstring = "20231211"
    if dateSubstring+"_mdb" in enlace["data-downloadurl"]:
        print(enlace["data-downloadurl"])


enlaces: [<a class="wpdm-download-link download-on-click btn btn-primary btn-xs" data-downloadurl="https://ar.kairosweb.com/download/bd_20231212_mdb_provisorio-zip/?wpdmdl=25338&amp;refresh=65773c309f89d1702313008" href="#" rel="nofollow">Descargar</a>, <a class="wpdm-download-link download-on-click btn btn-primary btn-xs" data-downloadurl="https://ar.kairosweb.com/download/bd_20231212_provisorio-zip/?wpdmdl=25337&amp;refresh=65773c30a13821702313008" href="#" rel="nofollow">Descargar</a>, <a class="wpdm-download-link download-on-click btn btn-primary btn-xs" data-downloadurl="https://ar.kairosweb.com/download/bd_20231211_mdb-zip/?wpdmdl=25335&amp;refresh=65773c30a2ac81702313008" href="#" rel="nofollow">Descargar</a>, <a class="wpdm-download-link download-on-click btn btn-primary btn-xs" data-downloadurl="https://ar.kairosweb.com/download/bd_20231211-zip/?wpdmdl=25334&amp;refresh=65773c30a423c1702313008" href="#" rel="nofollow">Descargar</a>, <a class="wpdm-download-link download-on-cli

## DOWNLOAD DB

In [9]:
urlDB = "https://ar.kairosweb.com/download/bd_"+dateSubstring+"_mdb-zip/?wpdmdl=25335&refresh=6577346c453481702311020"

response = sesion.get(urlDB)

if response.status_code == 200:
    with open(dateSubstring+"_mdb"+".zip", "wb") as f:
        f.write(response.content)
    print("downloaded")

downloaded


## UNZIP FILE

In [10]:
UnzipRute = dateSubstring+"_mdb"+".zip"
DestinyFolder = "./db/"
with zipfile.ZipFile(UnzipRute, 'r') as zip_ref:
    zip_ref.extractall(DestinyFolder)
    print("unzipped")

unzipped


## CONNECT TO ACCESS DB

In [11]:
print("dateSubstring", dateSubstring)
conn_str = (
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\sanmi\OneDrive\Desktop\SC\LARK\TEST\db\bd_'+dateSubstring+r'.mdb;'
)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

cursor.execute("SELECT * FROM prc WHERE ClaveProducto=8545 and ClavePresentacion=3")

for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()


dateSubstring 20231211
(8545, 3, '7384.66', datetime.datetime(2023, 12, 9, 0, 0))
